In [17]:
import pandas as pd
import numpy as np
from pmdarima import auto_arima
from joblib import Parallel, delayed
from utils import clean_daily_series, rmse, mae, smape, mase, directional_accuracy, print_evaluation_table

In [18]:
daily_train = pd.read_csv('../src/data/m4_forecasting/Daily-train.csv')
daily_test = pd.read_csv('../src/data/m4_forecasting/Daily-test.csv')

In [19]:
# M: seasonal period (7 for daily series (repeats weekly))
# H: forecast horizon (if 7, will predict 7 days into the future)
def forecast_daily_series(row, M=7, H=7):
    # Fit ARIMA
    model = auto_arima(
        row,
        seasonal=True,
        m=M,
        error_action='ignore',
        suppress_warnings=True
    )

    forecast = model.predict(n_periods=H)

    return forecast.tolist()


In [20]:

def evaluate_forecast(train_row, test_row, H=14):

    train_ts = clean_daily_series(train_row)
    test_ts = clean_daily_series(test_row)

    H_eval = min(H, len(test_ts))

    y_pred = forecast_daily_series(train_ts, H=H_eval)
    y_pred = np.atleast_1d(y_pred)
    y_true = np.atleast_1d(test_ts)
    return (
        rmse(y_true[:H_eval], y_pred[:H_eval]),
        mae(y_true[:H_eval], y_pred[:H_eval]),
        smape(y_true[:H_eval], y_pred[:H_eval]),
        mase(y_true[:H_eval], y_pred[:H_eval], train_ts, m=7),
        directional_accuracy(y_true[:H_eval], y_pred[:H_eval])
    )

In [ ]:
from sklearn.utils import shuffle


horizons = [1, 7, 14]
L = 30
NUM_SERIES = 50

daily_train_copy = daily_train.copy()
daily_test_copy = daily_test.copy()

daily_train_shuffled, daily_test_shuffled = shuffle(daily_train_copy, daily_test_copy, random_state=42)

all_results = {}

for H in horizons:
    print(f"Evaluating horizon H={H}")
    results = Parallel(n_jobs=-1, backend="loky", verbose=10)(
        delayed(evaluate_forecast)(daily_train_shuffled.iloc[i], daily_test_shuffled.iloc[i], H=H)
        for i in range(NUM_SERIES)
    )
    all_results[H] = results



Evaluating horizon H=1


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
python(98539) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(98540) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(98541) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(98542) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(98543) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(98544) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(98545) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(98546) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   10.5s
[Parallel(n_jobs

Evaluating horizon H=7


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   45.6s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:  2.5min remaining:   32.8s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:  3.4min remaining:   13.2s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  4.1min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Evaluating horizon H=14


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   34.3s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:  2.3min remaining:   30.8s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:  2.9min remaining:   11.0s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  3.5min finished


In [22]:
for H, results in all_results.items():
    # results: list of tuples per series
    rmses, maes, smapes, mases, das = zip(*results)

    print(f"\nHorizon: {H}")
    print_evaluation_table(rmses, maes, smapes, mases, das)



Horizon: 1
      Metric     Mean   Median
0       RMSE  55.2285  16.4000
1        MAE  55.2285  16.4000
2  sMAPE (%)   1.1565   0.4184
3       MASE   0.4169   0.2359
4         DA      NaN      NaN

Horizon: 7
      Metric      Mean   Median
0       RMSE  143.7468  52.9111
1        MAE  127.9598  42.2947
2  sMAPE (%)    2.7563   1.2746
3       MASE    1.0255   0.6384
4         DA    0.3900   0.5000

Horizon: 14
      Metric      Mean   Median
0       RMSE  178.0986  83.2642
1        MAE  154.2560  74.1695
2  sMAPE (%)    3.6061   1.6466
3       MASE    1.4291   0.8378
4         DA    0.3892   0.4231


/Users/willarachelian/Desktop/492 shared/knn-tree_layers-financial-forecasting/experiments/utils.py:93: RuntimeWarning: Mean of empty slice
  np.nanmean(das)
/Users/willarachelian/Desktop/492 shared/knn-tree_layers-financial-forecasting/experiments/utils.py:100: RuntimeWarning: All-NaN slice encountered
  np.nanmedian(das)
